In [1]:
import numpy as np
import sys
import os
import glob
import librosa
%matplotlib inline

# define dct-function like Hanke et al.

def dct_II(N):
   
   #Create N x N matrix of discrete cosine transform coefficients
   #Uses DCT-II formula:https://en.wikipedia.org/wiki/Discrete_cosine_transform#DCT-II

   d = np.array([np.cos(np.pi / N * (np.arange(N) + 0.5) * k)
                 for k in np.arange(N)],
                dtype='f4')
   d[0] *= 1 / np.sqrt(2)
   d *= np.sqrt(2.0 / N)
   return d

In [2]:
y, sr = librosa.load('/home/klara/Dokumente/7/Bachelorarbeit/Stimuli/normalized-deephouse2.mp3')
mfs = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=48)
chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr)

In [21]:
chroma_cqt

array([[ 0.6084895 ,  0.63929921,  0.54731728, ...,  0.96477041,
         1.        ,  1.        ],
       [ 0.74567038,  1.        ,  0.67094907, ...,  0.64261056,
         0.75916852,  0.71408911],
       [ 0.75810371,  0.67745213,  0.76874623, ...,  0.61294782,
         0.67371492,  0.47210393],
       ..., 
       [ 0.37598358,  0.29336314,  0.60786528, ...,  0.97713366,
         0.68834734,  0.5910989 ],
       [ 0.40669147,  0.19380606,  0.45620368, ...,  0.91527873,
         0.62670729,  0.36356125],
       [ 0.50786299,  0.33129451,  0.5758511 , ...,  1.        ,
         0.81125625,  0.60987434]])

In [20]:
mfs

array([[  6.15963622e+00,   3.74368670e+00,   1.56456221e+00, ...,
          4.93478416e+00,   7.47263618e+00,   5.27623551e+00],
       [  1.90793922e+00,   1.06674963e+00,   9.43084747e+00, ...,
          6.80476296e-01,   9.00468510e-01,   1.07009051e+00],
       [  1.35076817e-01,   4.64661283e-01,   1.41508880e+01, ...,
          2.90926517e-01,   1.13759673e-01,   5.08070582e-02],
       ..., 
       [  7.00719853e-05,   1.09148414e-04,   2.38940031e-03, ...,
          7.99555056e-04,   1.30027126e-03,   7.87792642e-04],
       [  2.21826607e-02,   2.80042598e-02,   9.48212224e-03, ...,
          3.18908136e-03,   2.42207868e-02,   1.95330488e-02],
       [  1.92193875e-02,   2.44891765e-02,   7.63208548e-03, ...,
          2.57242096e-03,   2.16001171e-02,   1.72222249e-02]])

In [3]:
x, sr2 = librosa.load('/home/klara/Dokumente/7/Bachelorarbeit/Stimuli/normalized-classic3.mp3')
mfs2 = librosa.feature.melspectrogram(y=x, sr=sr2, n_mels=48)
chroma_cqt2 = librosa.feature.chroma_cqt(y=x, sr=sr2)

In [22]:
mfs2

array([[  1.29259991e-01,   1.09361624e-01,   9.80351796e-02, ...,
          2.83839384e-01,   4.74474844e-01,   5.35345873e-01],
       [  4.82201289e-03,   4.72926483e-03,   6.70936515e-03, ...,
          1.65945686e-01,   3.46813819e-01,   5.65773201e-01],
       [  9.68745688e-04,   1.03149604e-03,   1.23642085e-03, ...,
          1.21004315e+00,   1.58422164e+00,   1.07627024e+00],
       ..., 
       [  2.81401287e-06,   1.04535819e-06,   4.77497668e-07, ...,
          9.74676793e-05,   1.24540094e-04,   1.47110034e-04],
       [  5.64252682e-06,   1.72832366e-06,   4.94658251e-07, ...,
          6.61491928e-05,   9.09483697e-05,   1.54068047e-04],
       [  6.60606521e-06,   1.74925309e-06,   2.43616646e-07, ...,
          2.83555710e-05,   5.37731401e-05,   2.46755034e-04]])

In [23]:
chroma_cqt2

array([[ 1.        ,  1.        ,  1.        , ...,  0.5377779 ,
         0.67345596,  0.82688501],
       [ 0.70250112,  0.7431346 ,  0.83270626, ...,  0.5209039 ,
         0.56605506,  0.56734795],
       [ 0.06552794,  0.10073647,  0.27241021, ...,  0.25437794,
         0.26693199,  0.27317147],
       ..., 
       [ 0.2394903 ,  0.23872239,  0.20052563, ...,  0.35659496,
         0.56530261,  0.57497872],
       [ 0.15273986,  0.12450547,  0.18844914, ...,  0.25275501,
         0.25036966,  0.37056886],
       [ 0.19930033,  0.20474348,  0.22357566, ...,  0.1781004 ,
         0.26266282,  0.33767301]])

In [4]:
nBands = 48
nCoefs = 13
DCT = dct_II(nBands)

In [5]:
# deephouse
mfcc = np.dot(DCT, 20 * np.log10(mfs + np.finfo(float).eps))
lq_mfs = np.dot(DCT[:nCoefs].T, mfcc[:nCoefs])
lq_mfs_norm = 10 ** (lq_mfs / 20.)
hq_mfs = np.dot(DCT[nCoefs:].T, mfcc[nCoefs:])
hq_mfs_norm = 10 ** (hq_mfs / 20.)

In [24]:
mfcc

array([[ -2.31366167e+02,  -2.37454116e+02,  -2.03278059e+02, ...,
         -2.33444424e+02,  -2.02618865e+02,  -2.49197368e+02],
       [  1.11755673e+02,   1.06045691e+02,   9.32356029e+01, ...,
          1.15276404e+02,   1.00185763e+02,   9.13090931e+01],
       [  1.93880602e+01,   2.78817254e+01,   3.63872074e+01, ...,
          2.96771190e+01,   4.00855305e+01,   5.11044326e+01],
       ..., 
       [  3.63295067e+00,   1.90596975e+00,  -1.62345823e+00, ...,
         -1.65350764e+00,  -1.96944089e+00,  -2.85044491e+00],
       [ -4.35909238e+00,  -5.37374320e+00,  -1.30027175e+00, ...,
         -1.13723626e+00,  -5.95350227e-01,   1.77964789e-01],
       [ -3.42013122e+00,  -1.94509691e+00,   2.41829033e+00, ...,
          3.62028524e+00,   2.90066204e+00,   3.14681576e+00]])

In [25]:
lq_mfs

array([[ 10.09033437,   9.32174481,  17.56383527, ...,   5.01598794,
          6.58679978,   5.08040517],
       [  2.59360897,   3.03925467,  14.09604178, ...,   2.87126517,
          3.32454233,   0.29393168],
       [ -9.33137722,  -7.03104451,   8.19503032, ...,  -0.70424355,
         -1.83410039,  -7.21254266],
       ..., 
       [-53.85498941, -52.3607173 , -47.2255546 , ..., -56.14004359,
        -45.3043727 , -48.37177071],
       [-41.70131299, -40.1778505 , -42.69531131, ..., -53.1331273 ,
        -37.15148901, -39.27091532],
       [-33.31609142, -31.75449009, -39.40174679, ..., -50.87730011,
        -31.38292784, -32.82437555]])

In [ ]:
hq_mfs

In [6]:
# classic
mfcc2 = np.dot(DCT, 20 * np.log10(mfs2 + np.finfo(float).eps))
lq_mfs2 = np.dot(DCT[:nCoefs].T, mfcc2[:nCoefs])
lq_mfs2_norm = 10 ** (lq_mfs2 / 20.)
hq_mfs2 = np.dot(DCT[nCoefs:].T, mfcc2[nCoefs:])
hq_mfs2_norm = 10 ** (hq_mfs2 / 20.)

In [7]:
# mahalanobis distances
import sklearn

# reshape arrays to make distance calculation possible
lq_mfs_reshape = lq_mfs.reshape(-1, 1)
lq_mfs2_reshape = lq_mfs2.reshape(-1, 1)
hq_mfs_reshape = hq_mfs.reshape(-1, 1)
hq_mfs2_reshape = hq_mfs2.reshape(-1, 1)
mfs_reshape = mfs.reshape(-1, 1)
mfs2_reshape = mfs2.reshape(-1, 1)
chroma_cqt_reshape = chroma_cqt.reshape(-1, 1)
chroma_cqt2_reshape = chroma_cqt2.reshape(-1, 1)

dist_timbre = sklearn.metrics.pairwise.pairwise_distances(lq_mfs_reshape, lq_mfs2_reshape, metric="mahalanobis")
dist_pitch = sklearn.metrics.pairwise.pairwise_distances(hq_mfs_reshape, hq_mfs2_reshape, metric="mahalanobis")
dist_spectrum = sklearn.metrics.pairwise.pairwise_distances(mfs_reshape, mfs2_reshape, metric="mahalanobis")
dist_chroma = sklearn.metrics.pairwise.pairwise_distances(chroma_cqt_reshape, chroma_cqt2_reshape, metric="mahalanobis")

In [8]:
dist_timbre

array([[  1.56169238e+00,   1.64352761e+00,   1.60076709e+00, ...,
          3.56935785e+00,   3.39259452e+00,   3.09683807e+00],
       [  1.53370250e+00,   1.61553773e+00,   1.57277721e+00, ...,
          3.54136796e+00,   3.36460463e+00,   3.06884819e+00],
       [  1.83385639e+00,   1.91569162e+00,   1.87293110e+00, ...,
          3.84152185e+00,   3.66475852e+00,   3.36900208e+00],
       ..., 
       [  6.58578401e-01,   5.76743175e-01,   6.19503697e-01, ...,
          1.34908706e+00,   1.17232373e+00,   8.76567287e-01],
       [  5.13521316e-02,   1.33187358e-01,   9.04268358e-02, ...,
          2.05901760e+00,   1.88225426e+00,   1.58649782e+00],
       [  1.14136369e-03,   8.06938625e-02,   3.79333405e-02, ...,
          2.00652410e+00,   1.82976077e+00,   1.53400432e+00]])

In [16]:
dist_timbre.shape

(12432, 12432)

In [9]:
mean_timbre = np.mean(dist_timbre)
mean_timbre

1.2289989171691584

In [10]:
dist_pitch

array([[ 1.17746975,  1.27791381,  1.0096363 , ...,  1.102163  ,
         1.01314765,  0.36733036],
       [ 1.62671858,  1.72716264,  1.45888513, ...,  0.65291417,
         0.56389881,  0.08191847],
       [ 3.62508041,  3.72552447,  3.45724696, ...,  1.34544766,
         1.43446302,  2.0802803 ],
       ..., 
       [ 2.01326162,  2.11370568,  1.84542817, ...,  0.26637113,
         0.17735578,  0.46846151],
       [ 2.14110576,  2.24154982,  1.97327231, ...,  0.13852699,
         0.04951163,  0.59630566],
       [ 2.20753441,  2.30797847,  2.03970096, ...,  0.07209835,
         0.01691701,  0.6627343 ]])

In [17]:
dist_pitch.shape

(12432, 12432)

In [11]:
mean_pitch = np.mean(dist_pitch)
mean_pitch

1.0735831819955963

In [12]:
dist_spectrum

array([[  5.03478075e-01,   5.05139396e-01,   5.06085044e-01, ...,
          5.14267666e-01,   5.14265544e-01,   5.14249432e-01],
       [  3.01769663e-01,   3.03430984e-01,   3.04376632e-01, ...,
          3.12559254e-01,   3.12557132e-01,   3.12541020e-01],
       [  1.19833850e-01,   1.21495171e-01,   1.22440819e-01, ...,
          1.30623441e-01,   1.30621319e-01,   1.30605207e-01],
       ..., 
       [  1.05771864e-02,   8.91586535e-03,   7.97021680e-03, ...,
          2.12404848e-04,   2.10282726e-04,   1.94170605e-04],
       [  8.98855788e-03,   7.32723680e-03,   6.38158825e-03, ...,
          1.80103340e-03,   1.79891128e-03,   1.78279915e-03],
       [  9.35406953e-03,   7.69274844e-03,   6.74709990e-03, ...,
          1.43552175e-03,   1.43339963e-03,   1.41728751e-03]])

In [18]:
dist_spectrum.shape

(12432, 12432)

In [13]:
mean_spectrum = np.mean(dist_spectrum)
mean_spectrum

0.32158581575184092

In [14]:
dist_chroma

array([[ 1.47291573,  1.47291573,  1.47291573, ...,  1.61918231,
         1.30104698,  1.01884845],
       [ 1.35700541,  1.35700541,  1.35700541, ...,  1.73509262,
         1.4169573 ,  1.13475876],
       [ 1.70305394,  1.70305394,  1.70305394, ...,  1.38904409,
         1.07090877,  0.78871024],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  3.09209803,
         2.77396271,  2.49176417],
       [ 0.71007966,  0.71007966,  0.71007966, ...,  2.38201837,
         2.06388305,  1.78168451],
       [ 1.46770576,  1.46770576,  1.46770576, ...,  1.62439227,
         1.30625695,  1.02405841]])

In [19]:
dist_chroma.shape

(3108, 3108)

In [15]:
mean_chroma = np.mean(dist_chroma)
mean_chroma

1.1608574499629196

In [ ]:
matrix_timbre = {(0, 3): 1, (2, 1): 2, (4, 3): 3}